In [1]:
from llama_index.llms.bedrock import Bedrock

# llm = Bedrock(
#     model="amazon.titan-text-express-v1",
#     aws_access_key_id="AWS Access Key ID to use",
#     aws_secret_access_key="AWS Secret Access Key to use",
#     aws_session_token="AWS Session Token to use",
#     aws_region_name="AWS Region to use, eg. us-east-1",
# )

llm = Bedrock(model="anthropic.claude-3-sonnet-20240229-v1:0")      # amazon.titan-text-express-v1

response = llm.complete("Paul Graham is ")

In [2]:
print(response)

Paul Graham is a computer scientist, entrepreneur, venture capitalist, and author. He is best known for the following:

1. Co-founding Y Combinator: Y Combinator is a prominent startup accelerator based in Silicon Valley. It has funded and helped launch thousands of startups, including Airbnb, Dropbox, Stripe, and Reddit.

2. Writing essays on startups and technology: Graham has written numerous influential essays on topics related to startups, programming, and entrepreneurship. His essays are widely read and have helped shape the thinking of many entrepreneurs and technologists.

3. Developing programming languages: Graham is the principal designer of the programming languages Arc and Bel. He has also worked on the development of other programming languages, such as Lisp and Prolog.

4. Authoring books: Graham has written several books, including "Hackers & Painters: Big Ideas from the Computer Age," "ANSI Common Lisp," and "On Lisp."

5. Advocating for the growth of startup ecosystem

In [65]:
from IPython.display import Markdown, display

text = """
Hi Alex, 
könntest du bitte für diesen Termin den großen Besprechungsraum 230 buchen.
 
Danke dir!
 
Viele Grüße,
Luigi"""


prompt = f"""
\n\nHuman: Please act as clerk, named Alex, in a big insurance company, Fantasy Insurance SE, who is only responsible for room reservations. 

Please answer all room booking requests in German and in a friendly way. 
<booking request>
{text}
</booking request>

Please structure your answer in the following markdown format: 
- Greeting
- Confirmation of the booking
- Additional information
- Farewell
\n\nAssistant:
"""

print(prompt)




Human: Please act as clerk, named Alex, in a big insurance company, Fantasy Insurance SE, who is only responsible for room reservations. 

Please answer all room booking requests in German and in a friendly way. 
<booking request>

Hi Alex, 
könntest du bitte für diesen Termin den großen Besprechungsraum 230 buchen.
 
Danke dir!
 
Viele Grüße,
Luigi
</booking request>

Please structure your answer in the following markdown format: 
- Greeting
- Confirmation of the booking
- Additional information
- Farewell


Assistant:



https://github.com/aws-samples/amazon-bedrock-samples/blob/main/multimodal/Claude3/Claude3-Sonnet-Multimodal-Example.ipynb

In [72]:
import json
import boto3
from botocore.config import Config

def generate_message(bedrock_runtime, model_id, messages, **param):
    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "messages": messages,
            **param
        }  
    )  
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
    return response_body

In [73]:
# Multimodal message (here use only text, no image as input):
input_message = [
    { "role": "user",
      "content": [
    #   {"type": "image","source": { "type": "base64","media_type":"image/jpeg","data": content_image}},
      {"type": "text", "text": prompt}
      ]
    }
]

In [74]:
boto_config = Config(retries={"max_attempts": 8})

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime", config=boto_config
)

param = {
        "max_tokens": 512,
        "temperature": 0.5,
        "top_p": 0.9,
    }

response_body = generate_message(bedrock_runtime, model_id = "anthropic.claude-3-sonnet-20240229-v1:0", messages=input_message, **param)

display(Markdown(text))
display(Markdown(response_body["content"][0]["text"]))


Hi Alex, 
könntest du bitte für diesen Termin den großen Besprechungsraum 230 buchen.
 
Danke dir!
 
Viele Grüße,
Luigi

- Guten Tag, Luigi!
- Ich habe den großen Besprechungsraum 230 für Ihren Termin reserviert.
- Bitte lassen Sie mich wissen, falls Sie noch etwas benötigen. Ich stehe Ihnen gerne zur Verfügung.
- Einen schönen Tag noch!

In [2]:
# llama-parse is async-first, running the sync code in a notebook requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

In [8]:
from llama_index.core import Settings

#Settings.llm = Bedrock(model="anthropic.claude-3-sonnet-20240229-v1:0")
Settings.llm = Bedrock(model="anthropic.claude-v2", temperature=0, max_tokens=3000)
#Settings.embed_model = embed_model
Settings.chunk_size = 512

In [3]:
import os
from llama_parse import LlamaParse

#os.environ['LLAMA_CLOUD_API_KEY'] = "LL-rVQIJc6wfFfriCohJI2Fbslnw8XZy4EUqVsIsK1oQx88jj921uCoqqH4nf268qwW"

documents = LlamaParse(result_type="markdown").load_data("./data/Agile_Grundlagen_2024_handout_3.pdf")

Error while parsing the file './data/Agile_Grundlagen_2024_handout_3.pdf': Failed to parse the file: {"detail":"Invalid authentication token"}


In [4]:
from llama_index.core.prompts import PromptTemplate
from pydantic.v1 import BaseModel

class Test(BaseModel):
    """My test class."""
    name: str

prompt = PromptTemplate("Please predict a Test with a random name related to {topic}.")

output = await llm.astructured_predict(Test, prompt, topic="cats")
print(output.name)

Fluffy Claws Test


In [4]:
from llama_index.embeddings.bedrock import BedrockEmbedding

In [5]:
embed_model = BedrockEmbedding()

In [6]:
embedding = embed_model.get_text_embedding("hello world")

In [7]:
from llama_index.embeddings.bedrock import BedrockEmbedding
import json

supported_models = BedrockEmbedding.list_supported_models()
print(json.dumps(supported_models, indent=2))

{
  "amazon": [
    "amazon.titan-embed-text-v1",
    "amazon.titan-embed-g1-text-02",
    "cohere.embed-english-v3",
    "cohere.embed-multilingual-v3"
  ],
  "cohere": [
    "amazon.titan-embed-text-v1",
    "amazon.titan-embed-g1-text-02",
    "cohere.embed-english-v3",
    "cohere.embed-multilingual-v3"
  ]
}


In [8]:
#embed_model = BedrockEmbedding(model="cohere.embed-multilingual-v3")
embed_model = BedrockEmbedding(model="amazon.titan-embed-text-v1")

coherePayload = [
    "This is a test document",
    "తెలుగు అనేది ద్రావిడ భాషల కుటుంబానికి చెందిన భాష.",
    "Esto es una prueba de documento multilingüe.",
    "攻殻機動隊",
    "Combien de temps ça va prendre ?",
    "Документ проверен",
]
embeddings = embed_model.get_text_embedding_batch(coherePayload)
print(embeddings)

[[0.65234375, -0.38085938, -0.29882812, 0.061523438, -0.375, 0.27929688, -0.0038909912, -0.0005607605, 0.10449219, -0.12890625, 0.5078125, 0.17773438, 0.15722656, 0.19628906, -0.9453125, -0.08496094, 0.29492188, -0.12402344, -0.10839844, 0.30078125, -0.45117188, 0.66796875, -0.40234375, 0.796875, 0.32617188, 0.2109375, 0.38671875, 0.1484375, 0.010253906, -0.44726562, -0.64453125, 0.57421875, 0.4140625, -1.03125, 0.17773438, -0.2421875, 0.19042969, 0.171875, 0.33398438, 0.26953125, -0.23828125, -0.08203125, 0.45117188, -0.20214844, -0.35742188, -0.123535156, 0.24609375, 0.057861328, 0.58984375, 0.671875, -0.12060547, 0.91796875, 0.5, -0.17089844, 0.15625, -0.12060547, -0.4296875, 0.04638672, 0.15429688, -0.50390625, 0.15332031, -0.53125, 0.0045776367, -0.625, -0.55078125, -0.10107422, -0.033691406, -0.21679688, -0.51953125, 0.640625, 0.15136719, -0.083496094, 0.07373047, 0.22460938, 0.265625, -0.036132812, 0.44726562, 0.11035156, 0.25585938, -0.85546875, -0.48828125, -0.09472656, -0.322

In [9]:
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

Settings.llm = Bedrock(model="anthropic.claude-v1")
Settings.embed_model = embed_model
Settings.chunk_size = 512

#reader = SimpleDirectoryReader(input_files=["./data/Agile_Grundlagen_2024_handout_3.pdf"])

# only load pdf files
required_exts = [".pdf"]

reader = SimpleDirectoryReader(input_dir="./data",
    required_exts=required_exts,
    recursive=True
    )

documents = reader.load_data()
print(f"Loaded {len(documents)} pages.")

Loaded 108 pages.


In [4]:
from llama_index.readers.file import UnstructuredReader

loader = UnstructuredReader()

documents = loader.load_data("./data/Agile_Grundlagen_2024_handout_3.pdf")
documents[0].text
documents[0].to_dict()

In [36]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=1000)

nodes = parser.get_nodes_from_documents(documents)
nodes[0].text

'16/04/2024\n1© Allianz Internal\nAgile \nGrundlagen\nDeutschlandAgile Grundlagen,  2024-lich\nWillkommen!\nInternal\nAgile Grundlagen\n“Das Einzige, was schlimmer ist, als Ihre Mitarbeiter zu schulen und\nsie gehen zu lassen, ist, sie nicht zu schulen und sie zu behalten. ”\nHenry Ford1\n2'

In [38]:
nodes[:5]

[TextNode(id_='420485ac-db18-48e5-8268-3885bae2ab1c', embedding=None, metadata={'page_label': '1', 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': '/Users/mn9lu5b/Library/CloudStorage/OneDrive-Allianz/Github_priv/rag_sandbox/data/Agile_Grundlagen_2024_handout_3.pdf', 'file_type': 'application/pdf', 'file_size': 3723800, 'creation_date': '2024-05-11', 'last_modified_date': '2024-04-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ce3d1e8e-6dd7-4f50-bb89-4cccfab3224c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': '/Users/mn9lu5b/Library/CloudStorage/OneDrive-Allianz/Github_priv/rag_sandbox/data/Agile_G

In [47]:
from unstructured.partition.pdf import partition_pdf

elements = partition_pdf("./data/Agile_Grundlagen_2024_handout_3.pdf", strategy='hi_res', include_page_breaks=True, infer_table_structure=False, languages=['deu'])    


This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name


In [29]:
# from unstructured.chunking.title import chunk_by_title

# chunks = chunk_by_title(elements, max_characters=1000)

# for z, chunk in enumerate(chunks):
#     print(f"Chunk {z}:\n", chunk)

In [16]:
from llama_index.core import Document

doc = Document(text="This is a test document.")
doc

Document(id_='978a9bf9-a777-403a-89de-f8e987fae41d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='This is a test document.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [58]:
from llama_index.core import Document

documents = []
meta = {}
for element in elements:
    metadata = element.metadata.to_dict()
    meta['page_label'] = metadata.get('page_number', None)
    meta["file_name"] = metadata["filename"]
    meta["file_path"] = metadata["file_directory"]
    meta["file_type"] = metadata["filetype"]
    documents.append(Document(text=element.text, metadata=meta))

documents[:5]

[Document(id_='e519177e-95af-41dd-a926-8af9b773ac9c', embedding=None, metadata={'page_label': 1, 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': './data', 'file_type': 'application/pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Internal  -lich Willkommen! ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='fc883572-46c6-4f76-8e4c-1a1d4d5fe4a8', embedding=None, metadata={'page_label': 1, 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': './data', 'file_type': 'application/pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='89d11f60-82c5-4840-82f5-c163080a6727', embedding=None, metadata={'page_

In [37]:
import chromadb
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore

chroma_client = chromadb.EphemeralClient()

chroma_collection = chroma_client.create_collection("Alex")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

documents = SimpleDirectoryReader(input_dir="./data",
    required_exts=required_exts,
    recursive=True
    ).load_data()

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

rag_application = index.as_query_engine()

In [10]:
index = VectorStoreIndex.from_documents(
    documents=documents,
    embed_model=embed_model
)

rag_application = index.as_query_engine()

In [12]:
from deepeval.integrations.llama_index import DeepEvalFaithfulnessEvaluator

# An example input to your RAG application
user_input = "What is this?"

# LlamaIndex returns a response object that contains
# both the output string and retrieved nodes
#response_object = rag_application.query(user_input)


In [ ]:
evaluator = DeepEvalFaithfulnessEvaluator()

evaluation_result = evaluator.evaluate_response(
    query=user_input, response=response_object
)
print(evaluation_result)

In [62]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/Users/mn9lu5b/miniforge3/envs/env_sandy/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
index = VectorStoreIndex.from_documents(
    documents=documents,
    embed_model=embed_model
)

rag_application = index.as_query_engine()

In [14]:
retriever = index.as_retriever(similarity_top_k=6)

In [15]:
question = "Was ist die Definition von agiler Arbeitsweise?"

contexts = retriever.retrieve(question)
contexts

[NodeWithScore(node=TextNode(id_='a5d9768e-8b60-4143-bb04-64c657a80898', embedding=None, metadata={'page_label': '14', 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': '/Users/mn9lu5b/Library/CloudStorage/OneDrive-Allianz/Github_priv/rag_sandbox/data/Agile_Grundlagen_2024_handout_3.pdf', 'file_type': 'application/pdf', 'file_size': 3723800, 'creation_date': '2024-05-11', 'last_modified_date': '2024-04-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1d9fca67-584f-4f26-a7d7-d6f1d14d1a65', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '14', 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': '/Users/mn9lu5b/Library/CloudStorage/OneDrive-Allianz/Github_priv/rag

In [16]:
context_list = [n.get_content() for n in contexts]
len(context_list)

10

In [17]:
context_list

['16/04/2024\n14© Allianz Internal\nDeutschlandAgile Grundlagen,  2024\n© Allianz Internal\nThe fundamentals ” / Mindset / Agile Werte & Prinzipien\nDeutschlandAgile Grundlagen,  2024What is agile about? (nach Craig Larman)\nQuelle:  „Scaling Lean & Agile Development: Thinking & Organizational Tools for Large-Scale Scrum ” von Craig LarmanAgile does not mean delivering faster.\nAgile does not mean fewer defects or higher quality.\nAgile does not mean higher productivity.\nAgile means agile -- the ability to move with quick easy \ngrace, to be nimble and adaptable. To embrace change and \nbecome master of change -- to complete through adaptability\nby being able to change faster and cheaper than your\ncompetition can.27\n28',
 '16/04/2024\n17© Allianz Internal\nWas ist Agile?\nDeutschlandAgile Grundlagen,  2024…in einer etwas realistischeren Darstellung\nQuelle: Bernd Östereich, oose Innovative Informatik\n© Allianz Internal\nWenn Agilit ät nicht das primäre Ziel verfolgt effektiver\nzu

In [22]:
from llama_index.core.query_engine import MultiStepQueryEngine
from llama_index.core.indices.query.query_transform.base import StepDecomposeQueryTransform

step_decompose_transform = StepDecomposeQueryTransform(llm=llm, verbose=True)

In [30]:
query_engine = index.as_query_engine(llm=llm)

query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    num_steps=3,
    query_transform=step_decompose_transform,
    index_summary="Used to answer questions about agile methodology."
)
response_gpt4 = query_engine.query(str_or_query_bundle = "Was ist die Definition von agiler Arbeitsweise?")

> Current query: Was ist die Definition von agiler Arbeitsweise?
> New query: Da der Kontext nur allgemeine Informationen über agile Methoden enthält und keine spezifischen Definitionen, ist es am besten, die ursprüngliche Frage beizubehalten:

"Was ist die Definition von agiler Arbeitsweise?"
> Current query: Was ist die Definition von agiler Arbeitsweise?
> New query: Da der Kontext nur allgemeine Informationen über agile Methoden enthält und keine spezifischen Definitionen, ist es am besten, die ursprüngliche Frage beizubehalten:

"Was ist die Definition von agiler Arbeitsweise?"
> Current query: Was ist die Definition von agiler Arbeitsweise?
> New query: Die ursprüngliche Frage lautete: 

Was ist die Definition von agiler Arbeitsweise?

Da der Kontext nur allgemeine Informationen über agile Methoden enthält und keine spezifischen Definitionen, ist es am besten, die ursprüngliche Frage beizubehalten:

"Was ist die Definition von agiler Arbeitsweise?"


ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the InvokeModel operation: Could not resolve the foundation model from the provided model identifier.

In [78]:
from llama_index.core import PromptTemplate

query_gen_str = """\
You are a helpful assistant that generates multiple search queries based on a \
single input query. Generate {num_queries} search queries, one on each line, \
related to the following input query:
Query: {query}
Queries:
"""
query_gen_prompt = PromptTemplate(query_gen_str)

llm = Bedrock(model="amazon.titan-text-express-v1")      # anthropic.claude-v2

def generate_queries(query: str, llm, num_queries: int = 4):
    response = llm.predict(
        query_gen_prompt, num_queries=num_queries, query=query
    )
    # assume LLM proper put each query on a newline
    queries = response.split("\n")
    queries_str = "\n".join(queries)
    print(f"Generated queries:\n{queries_str}")
    return queries

In [79]:
queries = generate_queries("Was ist die Definition von agiler Arbeitsweise?", llm)

Generated queries:
1. Was ist die Definition von agiler Arbeitsweise?
2. Was ist agile Arbeitsweise?
3. Was bedeutet agiler Arbeitsansatz?
4. Was ist agile Methoden?


In [80]:
queries

['1. Was ist die Definition von agiler Arbeitsweise?',
 '2. Was ist agile Arbeitsweise?',
 '3. Was bedeutet agiler Arbeitsansatz?',
 '4. Was ist agile Methoden?']

In [81]:
# import QueryBundle
from llama_index.core import QueryBundle

# import NodeWithScore
from llama_index.core.schema import NodeWithScore

# Retrievers
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    KeywordTableSimpleRetriever,
)

from typing import List

In [82]:
from llama_index.core import Settings

documents = SimpleDirectoryReader(input_dir="./data",
    required_exts=required_exts,
    recursive=True
    ).load_data()

nodes = Settings.node_parser.get_nodes_from_documents(documents)

In [83]:
from llama_index.core import StorageContext

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [84]:
from llama_index.core import SimpleKeywordTableIndex, VectorStoreIndex

vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

In [85]:
class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both semantic search and hybrid search."""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        keyword_retriever: KeywordTableSimpleRetriever,
        mode: str = "AND",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._keyword_retriever = keyword_retriever
        
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        keyword_nodes = self._keyword_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        keyword_ids = {n.node.node_id for n in keyword_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in keyword_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(keyword_ids)
        else:
            retrieve_ids = vector_ids.union(keyword_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [86]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# define custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=6)
keyword_retriever = KeywordTableSimpleRetriever(index=keyword_index)
custom_retriever = CustomRetriever(vector_retriever, keyword_retriever)

# define response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

# vector query engine
vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)
# keyword query engine
keyword_query_engine = RetrieverQueryEngine(
    retriever=keyword_retriever,
    response_synthesizer=response_synthesizer,
)

In [87]:
response = custom_query_engine.query("Was ist die Definition von agiler Arbeitsweise?")

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the InvokeModel operation: Could not resolve the foundation model from the provided model identifier.

In [ ]:
print(response)

[NodeWithScore(node=TextNode(id_='184a9414-ea5a-4b7f-aadb-e7ecde10a15e', embedding=None, metadata={'page_label': '14', 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': '/Users/mn9lu5b/Library/CloudStorage/OneDrive-Allianz/Github_priv/rag_sandbox/data/Agile_Grundlagen_2024_handout_3.pdf', 'file_type': 'application/pdf', 'file_size': 3723800, 'creation_date': '2024-05-11', 'last_modified_date': '2024-04-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9740a927-4d9b-4605-af25-a5daee000d3a', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '14', 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': '/Users/mn9lu5b/Library/CloudStorage/OneDrive-Allianz/Github_priv/rag

In [76]:
# in contrast, vector search will return an answer
response = vector_query_engine.retrieve("Was ist die Definition von agiler Arbeitsweise?")
response

[NodeWithScore(node=TextNode(id_='f003266d-c20a-4420-821e-98f2e70148e8', embedding=None, metadata={'page_label': '14', 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': '/Users/mn9lu5b/Library/CloudStorage/OneDrive-Allianz/Github_priv/rag_sandbox/data/Agile_Grundlagen_2024_handout_3.pdf', 'file_type': 'application/pdf', 'file_size': 3723800, 'creation_date': '2024-05-11', 'last_modified_date': '2024-04-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c686bff9-f365-4e41-9adf-55474b9048ac', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '14', 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': '/Users/mn9lu5b/Library/CloudStorage/OneDrive-Allianz/Github_priv/rag